In [ ]:
import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import load_model
import h5py
from tensorflow.keras import __version__ as keras_version
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation, Lambda, Cropping2D, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import GlobalMaxPooling2D

In [1243]:
import os
import csv
import cv2
import sys
import glob
import random
import collections
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
from tqdm import tqdm

from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn import model_selection

import xml.etree.ElementTree as ET

In [ ]:
# sys.path.append("../lib/cv")
# import GradientThresholds as GradThresh
# from GradientThresholds import GradientThresholds

# import importlib
# importlib.reload(GradThresh)

In [ ]:
class GradientThresholds:
    def __init__(self):
        self.img_m = None

    def apply_sobel_thresh(self, img, orient="x", ksize=(3,3), thresh=(0,255)):
        """
            Calculate directional gradient and identify pixels where the gradient
            falls within a particular threshold range for an image. Pass in image,
            choose orient to be x or y gradient, pick threshold min and max range
            to select for binary_output. The binary_output array is 1 where gradients
            are in threshold range, 0 everywhere else.
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        gray = cv2.GaussianBlur(gray, ksize, 0)

        if orient == "x":
            # apply sobel in "x" direction
            sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        elif orient == "y":
            sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        # Take abs value of derivative aka gradient
        abs_sobel = np.absolute(sobel)

        # scale result to 8-bit range (0-255)
        scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

        # apply lower and upper thresholds to mask scaled gradient
        binary_image = np.zeros_like(scaled_sobel)

        # apply 1's when scaled gradient is within threshold
        binary_image[(scaled_sobel >= thresh[0]) &
                     (scaled_sobel <= thresh[1])] = 1

        return binary_image

    # TODO: Create Gradient Magnitude Thresholding method
    def apply_grad_mag_thresh(self, img, sobel_kernel=3, mag_thresh=(0, 255)):
        """
            Calculate gradient magnitude
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # take gradient in x and y separately
        sobelx  = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
        sobely  = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
        # calculate gradient magnitude in x and y direction
        grad_mag = np.sqrt( (sobelx**2) + (sobely**2) )
        # scale to 8-bit and convert to type=np.uint8
        scale_factor = np.max(grad_mag)/255
        scaled_grad_mag = (grad_mag/scale_factor).astype(np.uint8)
        # create a binary mask where mag thresholds are met
        binary_image = np.zeros_like(scaled_grad_mag)
        binary_image[(scaled_grad_mag >= mag_thresh[0]) &
                     (scaled_grad_mag <= mag_thresh[1])] = 1
        return binary_image
    
    # TODO: Create Gradient Direction Thresholding method
    def apply_grad_dir_thresh(self, img, sobel_kernel=3, dir_thresh=(0, np.pi/2)):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # take gradient in x and y separately
        sobelx  = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
        sobely  = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
        # abs value of x and y gradients
        abs_sobelx = np.absolute(sobelx)
        abs_sobely = np.absolute(sobely)
        # calculate the gradient direction
        dir_grad = np.arctan2(abs_sobely, abs_sobelx)
        
        # create binary image where direction thresholds are met
        binary_image = np.zeros_like(dir_grad)
        binary_image[(dir_grad >= dir_thresh[0]) & (dir_grad <= dir_thresh[1])] = 1
        return binary_image
    
    # TODO: Create Combined Thresholding method
    def apply_combined_thresh(self, combination_code, grad_x = None, grad_y = None, grad_mag = None, grad_dir = None):
        combined = np.zeros_like(grad_x)
        if combination_code == 0:
            combined[ (grad_x == 1) & (grad_mag == 1) ] = 1
        elif combination_code == 1:
            combined[ (grad_x == 1) & (grad_dir == 1) ] = 1
        elif combination_code == 2: 
            combined[ (grad_x == 1) & (grad_mag == 1) & (grad_dir == 1) ] = 1  
        elif combination_code == 3:
            combined[ (grad_x == 1) & (grad_y == 1) & 
                      (grad_mag == 1) & (grad_dir == 1) ] = 1             
        else:
            print("Error: Choose a supported code for combined gradient")

        # Return binary result from multiple thresholds
        return combined
    
    def save_img(self, dst_path, filename, dst_img):
        """
            Save gradient thresholded image using OpenCV
        """
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)

        plt.imsave(dst_path + filename, dst_img, cmap="gray")

    def visualize(self, src_title, orig_img, dst_title, binary_img):
        """
            Visualize gradient thresholded image
        """
        img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,9))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title(src_title, fontsize=50)
        ax2.imshow(binary_img, cmap="gray")
        ax2.set_title(dst_title, fontsize=50)
        plt.subplots_adjust(left=0, right=1, top=0.9, bottom=0.)


In [ ]:
class ColorThresholds:
    # Apply Grayscale Thresholding
    def apply_gray_thresh(self, img, thresh = (0, 255)):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binary_img = np.zeros_like(gray)
        binary_img[ (gray > thresh[0]) & (gray <= thresh[1]) ] = 1
        return binary_img
    
    # Thresholding individual RGB Color Channels
    def apply_r_thresh(self, img, thresh = (0, 255)):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        r_img = img[:,:,0]
        binary_img = np.zeros_like(r_img)
        binary_img[ (r_img >= thresh[0]) & (r_img <= thresh[1]) ] = 1
        return binary_img
    
    def apply_g_thresh(self, img, thresh = (0, 255)):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        g_img = img[:,:,1]
        binary_img = np.zeros_like(g_img)
        binary_img[ (g_img >= thresh[0]) & (g_img <= thresh[1]) ] = 1
        return binary_img        

    def apply_b_thresh(self, img, thresh = (0, 255)):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        b_img = img[:,:,2]
        binary_img = np.zeros_like(b_img)
        binary_img[ (b_img >= thresh[0]) & (b_img <= thresh[1]) ] = 1
        return binary_img  
    
    def apply_rgb_thresh(self, num_code, rgb_r = None, rgb_g = None, rgb_b = None):
        """
            Combine RGB Thresholding binary images based on the red, green and/or
            blue thresholds already applied, they set private variables that can be
            used in this method. Choose based on number code, which thresholds you'd
            combine:
            0: R Binary, G Binary
            1: R Binary, B binary
            2: G Binary, B Binary
            3: R Binary, G Binary, B Binary
        """
        combined = np.zeros_like(rgb_r)
        if num_code == 0:
            combined[ (rgb_r == 1) | (rgb_g == 1) ] = 1
        elif num_code == 1:
            combined[ (rgb_r == 1) & (rgb_b == 1) ] = 1
        elif num_code == 2: 
            combined[ (rgb_g == 1) & (rgb_b == 1) ] = 1  
        elif num_code == 3:
            combined[ ((rgb_r == 1) | (rgb_g == 1)) & (rgb_b == 1) ] = 1             
        else:
            print("Error: Choose a supported code for combined rgb")

        # Return binary result from multiple thresholds
        return combined
    
    # Thresholding individual HSL Color Channels
    def apply_h_thresh(self, img, thresh = (0, 255)):
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        h_img = hls[:,:,0]
        binary_img = np.zeros_like(h_img)
        binary_img[ (h_img >= thresh[0]) & (h_img <= thresh[1]) ] = 1
        return binary_img
    
    def apply_l_thresh(self, img, thresh = (0, 255)):
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        l_img = hls[:,:,1]
        binary_img = np.zeros_like(l_img)
        binary_img[ (l_img >= thresh[0]) & (l_img <= thresh[1]) ] = 1
        return binary_img
    
    def apply_s_thresh(self, img, thresh = (0, 255)):
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        s_img = hls[:,:,2]
        binary_img = np.zeros_like(s_img)
        binary_img[ (s_img >= thresh[0]) & (s_img <= thresh[1]) ] = 1
        return binary_img
    
    # Apply Combined HLS Thresholding
    def apply_hls_thresh(self, num_code, hls_h = None, hls_l = None, hls_s = None):
        """
            Combine HLS Thresholding binary images based on the hue, lightness
            and/or saturation thresholds already applied, they set private 
            variables that can be used in this method. Choose based on number 
            code, which thresholds you'd combine:
            # 0: H Binary, L Binary
            # 1: H Binary, S binary
            # 2: L Binary, S Binary
            # 3: H Binary, L Binary, S Binary  
        """
        combined = np.zeros_like(hls_h)
        if num_code == 0:
            combined[ (hls_h == 1) & (hls_l == 1) ] = 1
        elif num_code == 1:
            combined[ (hls_h == 1) | (hls_s == 1) ] = 1
        elif num_code == 2: 
            combined[ (hls_l == 1) & (hls_s == 1) ] = 1  
        elif num_code == 3:
            combined[ (hls_h == 1) | ((hls_s == 1) & (hls_l == 1)) ] = 1             
        else:
            print("Error: Choose a supported code for combined hls")

        # Return binary result from multiple thresholds
        return combined        
        
        
        h_binary = self.apply_h_thresh(img, thresh[0])
        l_binary = self.apply_l_thresh(img, thresh[1])
        s_binary = self.apply_s_thresh(img, thresh[2])
        combined = np.zeros_like(s_binary)
        combined[ (h_binary == 1) & (l_binary == 1) & (s_binary == 1) ] = 1
        return combined
    
    def save_img(self, dst_path, filename, dst_img):
        """
        Save gradient thresholded image using OpenCV
        """
        # If filepath doesn't exist, create it
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        
        # Save binary image resulting from gradient thresholding
        plt.imsave(dst_path + filename, dst_img, cmap = "gray")
        
    def visualize(self, src_title, orig_img, dst_title, binary_img):
        """
        Visualize color thresholded image
        """
        img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,9))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title(src_title, fontsize=50)
        ax2.imshow(binary_img, cmap = 'gray')
        ax2.set_title(dst_title, fontsize=50)
        plt.subplots_adjust(left=0, right=1, top=0.9, bottom=0.)

In [ ]:
def get_filepaths(basepath, remove_ext=False):
    files = []
    filenames = []
    for filename in os.listdir(basepath):
#         print("filename =", filename)
        if remove_ext is True:
            file_name, file_ext = filename.split(".")
#             print("file_name =", file_name)
#             print("file_ext =", file_ext)
            filepath = os.path.join(basepath, file_name)
            files.append(filepath)
            filenames.append(file_name)
        else:
            filepath = os.path.join(basepath, filename)
            files.append(filepath)
            filenames.append(filename)
    return files, filenames
    
# TODO: Move above code into this function
def get_filepaths_videonum_dirs(basepath):
    videonum_frame_filepaths = []
    videonum_frame_filenames = []
    # loop each video number dir getting each frame filepath per video
    for videonum_dir in os.listdir(basepath):
        videonum_dirpath = os.path.join(basepath, videonum_dir)
        videonum_dirfilepaths, videonum_dirfilenames = get_filepaths(videonum_dirpath, remove_ext=True)
        videonum_frame_filepaths.extend(videonum_dirfilepaths)
        videonum_frame_filenames.extend(videonum_dirfilenames)
    return videonum_frame_filepaths, videonum_frame_filenames
    
def get_filepaths_videonum_dirs_xy(X_basepath, y_basepath):
    videonum_X_filepaths = []
    videonum_X_filenames = []
    videonum_y_filepaths = []
    videonum_y_filenames = []
    for videonum_X_dir, videonum_y_dir in zip(os.listdir(X_basepath), os.listdir(y_basepath)):
        videonum_X_dirpath = os.path.join(X_basepath, videonum_X_dir)
        videonum_y_dirpath = os.path.join(y_basepath, videonum_y_dir)
        videonum_y_dirfilepaths_tmp = []
        videonum_y_dirfilenames_tmp = []
        videonum_X_dirfilepaths, videonum_X_dirfilenames = get_filepaths(videonum_X_dirpath, remove_ext=True)
        videonum_y_dirfilepaths, videonum_y_dirfilenames = get_filepaths(videonum_y_dirpath, remove_ext=True)
#         print("len(videonum_X_dirfilenames) =", len(videonum_X_dirfilenames))
#         print("len(videonum_y_dirfilenames) =", len(videonum_y_dirfilenames))
        if len(videonum_X_dirfilenames) != len(videonum_y_dirfilenames):
            for y_i in range(len(videonum_y_dirfilenames)):
                if videonum_y_dirfilenames[y_i] in videonum_X_dirfilenames:
#                     print("videonum_y_dirfilenames[y_i] =", videonum_y_dirfilenames[y_i])
#                     print("is in videonum_X_dirfilenames = ", videonum_X_dirfilenames)
                    videonum_y_dirfilenames_tmp.append(videonum_y_dirfilenames[y_i] + ".xml")
                    videonum_y_dirfilepaths_tmp.append(videonum_y_dirfilepaths[y_i] + ".xml")
            videonum_y_filepaths.extend(videonum_y_dirfilepaths_tmp)
            videonum_y_filenames.extend(videonum_y_dirfilenames_tmp)
#             print("len(videonum_X_dirfilenames) =", len(videonum_X_dirfilenames))
#             print("len(videonum_y_dirfilenames_tmp) =", len(videonum_y_dirfilenames_tmp))
        else:
            videonum_y_dirfilepaths = [filepath + ".xml" for filepath in videonum_y_dirfilepaths]
            videonum_y_dirfilenames = [filename + ".xml" for filename in videonum_y_dirfilenames]
            videonum_y_filepaths.extend(videonum_y_dirfilepaths)
            videonum_y_filenames.extend(videonum_y_dirfilenames)
#             print("len(videonum_X_dirfilenames) =", len(videonum_X_dirfilenames))
#             print("len(videonum_y_dirfilenames) =", len(videonum_y_dirfilenames))
        videonum_X_dirfilepaths = [filepath + ".jpg" for filepath in videonum_X_dirfilepaths]
        videonum_X_dirfilenames = [filename + ".jpg" for filename in videonum_X_dirfilenames]
        videonum_X_filepaths.extend(videonum_X_dirfilepaths)
        videonum_X_filenames.extend(videonum_X_dirfilenames)
    return videonum_X_filepaths, videonum_X_filenames, videonum_y_filepaths, videonum_y_filenames
    
def read_img(img_path, flag = cv2.IMREAD_COLOR):
    # (height, width, 3)
    image = cv2.imread(img_path, flag)
    return image

def get_xml_label_names(xml_files):
    label_names = []
    for xml_file in tqdm(xml_files):
        train_y_tree = ET.parse(xml_file)
        train_y_root = train_y_tree.getroot()
        if train_y_root.find("object") != None:
            train_y_object = train_y_root.find("object")
            train_y_polyp_name = train_y_object.find("name").text
        else:
            train_y_polyp_name = "Not Specified"
        label_names.append(train_y_polyp_name)
    return label_names

def get_xml_boundboxes(xml_files):
    bound_boxes = []
    for xml_file in tqdm(xml_files):
        train_y_tree = ET.parse(xml_file)
        train_y_root = train_y_tree.getroot()
        if train_y_root.find("object") != None:
            train_y_object = train_y_root.find("object")
            train_y_bndbox = train_y_object.find("bndbox")
            train_y_bndbox_xmin = train_y_bndbox.find("xmin").text
            train_y_bndbox_ymin = train_y_bndbox.find("ymin").text
            train_y_bndbox_xmax = train_y_bndbox.find("xmax").text
            train_y_bndbox_ymax = train_y_bndbox.find("ymax").text
            bound_box_tuple = (train_y_bndbox_xmin, train_y_bndbox_ymin, train_y_bndbox_xmax, train_y_bndbox_ymax)
        else:
            bound_box_tuple = False
        bound_boxes.append(
            bound_box_tuple
        )
    return bound_boxes

# normalizes image pixel values betwen -0.5 and 0.5
def normalize_images(images):
    norm_images = []
    for image in tqdm(images):
        norm_img = (image/255.0) - 0.5
        norm_images.append(norm_img)
    return norm_images

def normalize_image(image):
    norm_img = (image/255.0) - 0.5
    return norm_img

def resize_image(image, size):
    resized_image = cv2.resize(image, (size, size))
    return resize_image
    
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def get_images(image_group, size, flag = cv2.IMREAD_COLOR):
    images = []
    for image_path in tqdm(image_group):
        image = read_img(image_path, flag)
        resized_img = cv2.resize(image, (size, size))
        images.append(resized_img)
    return images

def remove_non_labeled_polyps(img_filepaths, img_filenames, polyp_names):
    cln_X_img_filepaths = []
    cln_X_img_filenames = []
    cln_y_polyp_names = []
    for img_filepath, img_filename, label in zip(img_filepaths, img_filenames, polyp_names):
        if label != "Not Specified":
            cln_X_img_filepaths.append(img_filepath)
            cln_X_img_filenames.append(img_filename)
            cln_y_polyp_names.append(label)
            
    return cln_X_img_filepaths, cln_X_img_filenames, cln_y_polyp_names

def create_image_label_groups(image_paths, image_labels, batch_size=32):
    # Divide image paths and image labels into groups of BATCH_SIZE
    image_groups = list()
    label_groups = list()
    
    for idx in range(0, len(image_paths), batch_size):
        image_batch = list()
        for index in range(idx, idx + batch_size):
            image_batch.append(image_paths[index % len(image_paths)])
        image_groups.append(image_batch)
            
    for idx in range(0, len(image_labels), batch_size):
        for index in range(idx, idx + batch_size):
            label_groups.append(image_labels[index % len(image_labels)])
    
    return image_groups, label_groups
    
def create_image_batch(image_group):
    # get the max image shape from the batch of images
    max_shape = tuple()
    image_shapes = list()
    
    for channel in range(3):
        for image in image_group:
            image_shapes.append(image.shape[channel])
        
def change_extension(img_file, file_type):
    p = Path(img_file)
    return p.rename(p.with_suffix(file_type))
    
# loads image from filepath using opencv
def get_image(basepath, filepath):
    # read images
    source_path = filepath
    # extract filename from filepath using split and check platform
    if sys.platform == 'win32':
        filename = source_path.split("\\")[-1]
    elif sys.platform == 'linux' or sys.platform == 'darwin':
        filename = source_path.split("/")[-1]
    # add filename to end of path to PolypSet dir, so platform isn't an issue
    img_path_on_fs = basepath + filename
    # load image using opencv
    image = cv2.imread(img_path_on_fs)
    return image


def plot_sample(X, y, index):
    plt.figure(figsize = (30,4))
    plt.imshow(X[index])
    plt.xlabel(classes[y[index]])

In [ ]:
sm.set_framework('tf.keras')

## Load Training Set Images & Masked Labels

In [ ]:
train_y_basepath = "PolypsSet/train2019/Annotation"
file_type = ".xml"
train_y_filepaths, train_y_filenames = get_filepaths(train_y_basepath, file_type)

In [ ]:
train_X_basepath = "PolypsSet/train2019/Image"
file_type = ".jpg"
train_X_filepaths, train_X_filenames = get_filepaths(train_X_basepath, file_type)

In [ ]:
train_y_polyp_names = get_xml_label_names(train_y_filepaths)

In [ ]:
plt.figure()
sns.countplot(train_y_polyp_names)
plt.title("Extracted Polyp Name Labels")

In [ ]:
cln_train_X_filepaths, cln_train_X_filenames, cln_train_y_polyp_names = remove_non_labeled_polyps(
    train_X_filepaths, train_X_filenames, train_y_polyp_names)

In [ ]:
len(cln_train_X_filenames)

In [ ]:
plt.figure()
sns.countplot(cln_train_y_polyp_names)
plt.title("Extracted Polyp Name Labels")

In [ ]:
len(cln_train_X_filepaths)

In [ ]:
len(cln_train_y_polyp_names)

In [ ]:
# read BGR cause of OpenCV: load 100 images
train_X_images = get_images(cln_train_X_filepaths[:100], 192)

In [ ]:
X_train = np.array(train_X_images)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train / 255

In [ ]:
label_enc = LabelEncoder()
train_y = label_enc.fit_transform(cln_train_y_polyp_names)
classes = ['adenomatous', 'hyperplastic']

In [ ]:
plt.figure()
sns.countplot(train_y)
plt.title("Label Encoded Polyps (0=Adenomatous; 1=Hyperplastic)")

In [ ]:
# np.save("polyp_y_train_27048.npy", train_y)

In [ ]:
# np.save("polyp_x_train_27048.npy", X_train)

## Feature Engineer Polyp Masked Labels For Training

Use color transforms, gradients, etc. to create a polyp masked threshold binary image per polyp image. After comparing masking for 2 images. I am going to try a labeling tool instead and label 100 to 1000 images.

Here is the link I am going to try, they have built a labeling tool using matplotlib for masking images as labels that will be used for training UNet:

- [How we built an easy-to-use image segmentation tool with transfer learning](https://towardsdatascience.com/how-we-built-an-easy-to-use-image-segmentation-tool-with-transfer-learning-546efb6ae98)

Check out **GitHub Repo: [mpl-interations](https://github.com/ianhi/mpl-interactions)**

Check out **GitHub Repo: [matplotlib/ipympl](https://github.com/matplotlib/ipympl#install-the-jupyterlab-extension)**

Check out **[mpl-interactions Tutorial: Image Segmentation Label Tool](https://mpl-interactions.readthedocs.io/en/latest/examples/image-segmentation.html)**

For using custom Matplotlib Image Segmenter tool in Jupyter Notebook, you can install it with pip:

~~~bash
pip install mpl_interactions["jupyter"] # will install necessary deps for using in jupyter

# for use outside jupyter
pip install mpl_interactions

# install ipympl; they recommend installing jupyterlab >= 3
pip install ipympl

# recommended
conda install -c conda-forge jupyterlab

# need to install jupyter labextension
conda install -c conda-forge nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib
~~~

## Draw Masked Polyp Labels with Matplotlib's Image Segmenter

Naming convention will be polyp type followed by the original image file number.

In [ ]:
%matplotlib ipympl
import matplotlib.cbook as cbook

from mpl_interactions import image_segmenter

In [ ]:
# last one: 99
polyp_img_num = 99

In [ ]:
train_X_img_filename = cln_train_X_filenames[polyp_img_num]
print(train_X_img_filename)

In [ ]:
train_y_polyp_name = cln_train_y_polyp_names[polyp_img_num]
print(train_y_polyp_name)

In [ ]:
# yellow for polyp
train_X_img_bgr = train_X_images[polyp_img_num]
train_X_img_rgb = cv2.cvtColor(train_X_img_bgr, cv2.COLOR_BGR2RGB)
segmenter = image_segmenter(train_X_img_rgb, 
                            mask_colors="yellow", 
                            mask_alpha=0.75,
                            figsize=(4, 4))
plt.axis("off")
display(segmenter)

In [ ]:
plt.figure()
plt.imshow(segmenter.mask)

In [ ]:
# I will get a saved img 192x192 since earlier I resized img to 192x192. Factor of 32 for obj detect
plt.imsave("PolypsSet/train2019/RGBSegLabel/" + train_X_img_filename, 
           segmenter.mask)

In [ ]:
plt.figure()
plt.imshow(segmenter.mask, cmap = "gray")

In [ ]:
plt.imsave("PolypsSet/train2019/GraySegLabel/" + train_X_img_filename, 
           segmenter.mask, cmap = "gray")

## Loading Masked Labels from Feature Engineering

In [ ]:
train_masks_basepath = "PolypsSet/train2019/GraySegLabel"
file_type = ".jpg"
train_masks_filepaths, train_masks_filenames = get_filepaths(train_masks_basepath, file_type)

In [ ]:
train_mask_images = get_images(train_masks_filepaths, 192, flag = cv2.IMREAD_GRAYSCALE)
y_train_masks = np.array(train_mask_images)
y_train_masks = y_train_masks/255

## Data Augment Images

In [1234]:
import albumentations as A

In [1235]:
images_to_generate = 2000

In [1239]:
img_augmented_path="PolypsSet/train2019/aug_img_192/"
msk_augmented_path="PolypsSet/train2019/aug_gray_mask_192/" # path to gray seg masks

In [1237]:
aug = A.Compose([
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=1),
    A.Transpose(p=1),
    A.GridDistortion(p=1)
])

In [1251]:
i=1

In [1252]:
# augmenting on top of 100 images and 100 masked images gen 2000
# then save them in a folder
while i <= images_to_generate:
    number = random.randint(0, len(train_X_images[:100])-1)
    orig_image = train_X_images[number]
    orig_mask = train_mask_images[number]
    
    augmented = aug(image = orig_image, mask = orig_mask)
    transformed_image = augmented["image"]
    transformed_mask = augmented["mask"]
    
    # due to opencv being bgr, so go rgb
    transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
    
    new_image_path = "%s/aug_image_%s.jpg" %(img_augmented_path, i)
    new_mask_path = "%s/aug_mask_%s.jpg" %(msk_augmented_path, i)
    
    plt.imsave(new_image_path, transformed_image)
    plt.imsave(new_mask_path, transformed_mask, cmap="gray")
    i = i+1

## Loading Validation Set

In [ ]:
valid_X_basepath = "PolypsSet/val2019/Image"
valid_y_basepath = "PolypsSet/val2019/Annotation"

# Problem is that the valid set has more labels than there are images, so need to make them equal
valid_X_filepaths, valid_X_filenames, valid_y_filepaths, valid_y_filenames= get_filepaths_videonum_dirs_xy(valid_X_basepath, valid_y_basepath)

In [ ]:
valid_y_polyp_names = get_xml_label_names(valid_y_filepaths)

In [ ]:
sns.countplot(valid_y_polyp_names)

In [ ]:
cln_valid_X_filepaths, cln_valid_y_polyp_names = remove_non_labeled_polyps(
    valid_X_filepaths, valid_y_polyp_names)

In [ ]:
sns.countplot(cln_valid_y_polyp_names)

In [ ]:
valid_X_images = get_images(cln_valid_X_filepaths, 192)

In [ ]:
X_valid = np.array(valid_X_images)

In [ ]:
X_valid = X_valid / 255

In [ ]:
valid_y = label_enc.fit_transform(cln_valid_y_polyp_names)

In [ ]:
np.save("polyp_x_valid_4214.npy", X_valid)

In [ ]:
np.save("polyp_y_valid_4214.npy", valid_y)

## Loading Testing Set

In [1275]:
test_X_basepath = "PolypsSet/test2019/Image"
test_y_basepath = "PolypsSet/test2019/Annotation"

# Potential Problem is that the test set has more labels than there are images, so need to make them equal
test_X_filepaths, test_X_filenames, test_y_filepaths, test_y_filenames= get_filepaths_videonum_dirs_xy(test_X_basepath, test_y_basepath)

In [1276]:
test_y_polyp_names = get_xml_label_names(test_y_filepaths)

100%|█████████████████████████████████████████████████████████████████████████████| 4872/4872 [00:25<00:00, 189.81it/s]


In [1277]:
sns.countplot(test_y_polyp_names)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:title={'center':'Test Polyp Img After UNet Segmentation'}, xlabel='adenomatous', ylabel='count'>

In [1278]:
cln_test_X_filepaths, cln_test_X_filenames, cln_test_y_polyp_names = remove_non_labeled_polyps(
    test_X_filepaths, test_X_filenames, test_y_polyp_names)

In [1279]:
# cln_test_X_filepaths[0]

In [1280]:
test_X_images = get_images(cln_test_X_filepaths, 192)

100%|█████████████████████████████████████████████████████████████████████████████| 4719/4719 [00:44<00:00, 105.43it/s]


In [1281]:
X_test = np.array(test_X_images)

In [1282]:
X_test = X_test / 255

In [1283]:
sns.countplot(cln_test_y_polyp_names)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:title={'center':'Test Polyp Img After UNet Segmentation'}, xlabel='adenomatous', ylabel='count'>

In [1284]:
test_y = label_enc.fit_transform(cln_test_y_polyp_names)

In [ ]:
np.save("polyp_y_test_4719.npy", test_y)

In [ ]:
np.save("polyp_x_test_4719.npy", X_test)

## Build UNet

In [ ]:
# N = X_train.shape[-1]

In [ ]:
# y_train = np.asarray(train_y).astype('float32').reshape((-1,1))

In [ ]:
# train_masks_filepaths

In [ ]:
# fig = plt.figure(figsize=(4,4))
# plt.imshow(train_mask_images[0], cmap = "gray")

In [ ]:
# y_train_masks[0]

In [ ]:
# y_train_masks[0]

In [1253]:
aug_train_X_basepath = "PolypsSet/train2019/aug_img_192"
file_type = ".jpg"
aug_train_X_filepaths, aug_train_X_filenames = get_filepaths(aug_train_X_basepath, file_type)

In [1254]:
aug_train_X_images = get_images(aug_train_X_filepaths, 192)

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:13<00:00, 152.04it/s]


In [1255]:
aug_X_train = np.array(aug_train_X_images)

In [1256]:
aug_X_train = aug_X_train / 255

In [1257]:
aug_train_masks_basepath = "PolypsSet/train2019/aug_gray_mask_192"
file_type = ".jpg"
aug_train_masks_filepaths, aug_train_masks_filenames = get_filepaths(aug_train_masks_basepath, file_type)

In [1258]:
aug_train_mask_images = get_images(aug_train_masks_filepaths, 192, flag = cv2.IMREAD_GRAYSCALE)
aug_y_train_masks = np.array(aug_train_mask_images)
aug_y_train_masks = aug_y_train_masks/255

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:12<00:00, 166.03it/s]


In [1259]:
#expand y by 1 dimension adding a 1 to the last position
aug_y_train_masks = np.expand_dims(aug_y_train_masks, axis=3)

In [1260]:
aug_y_train_masks.shape

(2000, 192, 192, 1)

In [1261]:
aug_X_train.shape

(2000, 192, 192, 3)

In [1262]:
X_train_polyp, X_test_polyp, y_train_polyp, y_test_polyp = model_selection.train_test_split(
    aug_X_train, aug_y_train_masks, test_size=0.2, random_state=42
)

In [1263]:
len(y_train_polyp)

1600

In [1264]:
len(y_test_polyp)

400

In [1265]:
# https://segmentation-models.readthedocs.io/en/latest/api.html
# Can take any img size, but must be divisible by factor of 32
model = Unet(backbone_name='resnet34', input_shape=(None,None,3), 
             classes=1, activation="sigmoid", encoder_weights='imagenet')

In [1271]:
# mean squared error since this is a pixel classification
# MSE is used to measure the difference between the source image and the segmented image,
# the smaller the value of RMSE, the better the segmentation performance
model.compile('Adam', loss=bce_jaccard_loss, metrics=["accuracy", iou_score])

In [1269]:
print(model.summary())

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d_170 (ZeroPaddin  (None, None, None,   0          ['bn_data[0][0]']                
 g2D)                           3)                                                         

In [1273]:
history = model.fit(
    x=X_train_polyp,
    y=y_train_polyp,
    batch_size=8,
    epochs=6,
    validation_data=(X_test_polyp, y_test_polyp),
    verbose=1
)

Epoch 1/6
200/200 [==============================] - 50s 222ms/step - loss: 0.4726 - accuracy: 0.9429 - iou_score: 0.6273 - val_loss: 0.5216 - val_accuracy: 0.9361 - val_iou_score: 0.6065
Epoch 2/6
200/200 [==============================] - 21s 103ms/step - loss: 0.4223 - accuracy: 0.9456 - iou_score: 0.6701 - val_loss: 0.5806 - val_accuracy: 0.9383 - val_iou_score: 0.5581
Epoch 3/6
200/200 [==============================] - 22s 112ms/step - loss: 0.4028 - accuracy: 0.9464 - iou_score: 0.6836 - val_loss: 0.6708 - val_accuracy: 0.9357 - val_iou_score: 0.5138
Epoch 4/6
200/200 [==============================] - 24s 121ms/step - loss: 0.3741 - accuracy: 0.9477 - iou_score: 0.7068 - val_loss: 0.5007 - val_accuracy: 0.9379 - val_iou_score: 0.6171
Epoch 5/6
200/200 [==============================] - 25s 126ms/step - loss: 0.3319 - accuracy: 0.9497 - iou_score: 0.7404 - val_loss: 0.5025 - val_accuracy: 0.9407 - val_iou_score: 0.6253
Epoch 6/6
200/200 [==============================] - 26s 129

In [1286]:
accuracy = model.evaluate(X_test[:100], y_test[:100])

ValueError: Data cardinality is ambiguous:
  x sizes: 100
  y sizes: 4
Make sure all arrays contain the same number of samples.

In [ ]:
# only trained on 20 polyp masked images so far
model.save("polyp_unet_bin_seg.h5")

## Plot Train vs Valid Loss Curves After Training UNet

In [ ]:
plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("UNet CNN Train and Valid Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Train", "Valid"], loc="upper right")
plt.show()

## UNet Predict Polyp Segmentation on Test Image

In [ ]:
# load model if already have unet trained model saved
# model = load_model("polyp_unet_bin_seg.h5")

In [ ]:
# use one of the test images
test_img_1_rgb = cv2.cvtColor(test_X_images[0], cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(test_img_1_rgb)
plt.xlabel(classes[0])
plt.title("Test Polyp Img Before UNet Segmentation")
# plt.show()

In [ ]:
test_img_1_rgb.shape

In [ ]:
# expand test img1 dimension adding 1 at front
test_img_1_rgb = np.expand_dims(test_img_1_rgb, axis=0)

In [ ]:
test_img_1_rgb.shape

In [ ]:
polyp_seg_pred = model.predict(test_img_1_rgb)

In [ ]:
polyp_seg_pred.shape

In [ ]:
polyp_seg_pred_img = polyp_seg_pred.reshape(y_train_masks[0].shape)

In [ ]:
polyp_seg_pred_img.shape

In [ ]:
# we can see unet segmentation doesnt capture the polyp. We need more training masked labels
plt.figure()
plt.imshow(polyp_seg_pred_img, cmap="gray")
plt.xlabel(classes[0])
plt.title("Test Polyp Img After UNet Segmentation")
# plt.imsave("unet/seg_pred_images/test0_segmented_v3.jpg", polyp_seg_pred_img, cmap = "gray")
plt.savefig("unet/trained_100/seg_pred_images/test0_segmented.jpg")

## References

- [Image Segmentation: Kumaravel Subramaniam Tamilselvan and Govindasamy Murugesan](https://cdn.intechopen.com/pdfs/60741.pdf)